In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os

In [2]:
def get_page_contents(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }

    page = requests.get(url, headers=headers)

    if page.status_code == 200:
        return page.text

    return None

In [3]:
def get_parcours_MathApp(page_contents):
    soup = BeautifulSoup(page_contents, 'html.parser')
    structured_content = {}
    
    # Find the main div containing the sections
    parcours_math_div = soup.find('div', id='elementor-tab-content-1301', class_='elementor-tab-content elementor-clearfix')
    
    if parcours_math_div:
        # Find all h3 headings within this div
        headings = parcours_math_div.find_all('h3')
        
        for heading in headings:
            key_text = heading.get_text(strip=True)
            
            # Handle 'Matières par semestre' specially
            if key_text == 'Matières par semestre':
                semester_content = {}
                
                for i in range(1, 7):  # Loop through semesters 1 to 6
                    semester_heading = heading.find_next(string=f'Semestre {i}')
                    
                    if semester_heading:
                        # Get the <ul> list that follows the semester heading
                        materials_list = semester_heading.find_next('ul')
                        materials = [li.get_text(strip=True) for li in materials_list.find_all('li')] if materials_list else []
                        
                        # Add to the dictionary for the current semester
                        semester_content[f'Semestre {i}'] = materials
                
                # Add the gathered semester content to the main dictionary under 'Matières par semestre'
                structured_content['Matières par semestre'] = semester_content
            
            else:
                # Collect both <p> and <ul> elements following the heading
                values = []
                for sibling in heading.find_next_siblings(['p', 'ul'], limit=4):  # Limit to avoid unrelated content
                    if sibling.name == 'p':
                        values.append(sibling.get_text(strip=True))
                    elif sibling.name == 'ul':
                        values.extend([li.get_text(strip=True) for li in sibling.find_all('li')])
                structured_content[key_text] = values
    
    return structured_content


In [4]:
if __name__ == '__main__':
    url = 'https://www.esb.tn/programmes/licences/licence-en-mathematiques-appliquees/'
    page_contents = get_page_contents(url)
    result=[]   #keep result when appending all of the programs for now we will try to use it for csv test file

    if page_contents:
        result.append(get_parcours_MathApp(page_contents))
    else:
        print('Failed to get page contents.')

In [5]:
field_names = [
    'Objectifs', 'Contenu', 'Compétences', 'Métiers', 'Secteurs d’activité', 'Partenariats professionnels',
    'Semestre 1', 'Semestre 2', 'Semestre 3', 'Semestre 4', 'Semestre 5', 'Semestre 6'
]

# Flatten the data
flattened_result = {}
for dict in result:
    for key, value in dict.items():
        if key == 'Matières par semestre':
            # Extract each semester as a separate field
            for semester, courses in value.items():
                flattened_result[semester] = ', '.join(courses)
        elif isinstance(value, list):
            # Join list items with commas
            flattened_result[key] = ', '.join(value)
        else:
            flattened_result[key] = value
print(flattened_result)
# Write to CSV
with open('./licence.csv', 'a', newline='', encoding='utf-8') as csvfile:
    # Check if the file is empty
    file_is_empty = os.stat('./licence.csv').st_size == 0

    # Create a DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    
    # Write the header only if the file is empty
    if file_is_empty:
        writer.writeheader()
    
    # Write the flattened result as a row
    writer.writerow(flattened_result)

{'Objectifs': 'Le monde est devenu mathématique. La décision repose sur l’analyse de données de plus en plus massives – Big Data., LaLicence en Mathématiques Appliquées à l’Analyse des Données et à l’Aide à la Décisionfait appel à des outils d’intelligence artificielle et de machine learning et à des compétences et savoir-faire mathématiques et statistiques., Cette licence s’adresse aux bacheliers en mathématiques, informatique, sciences expérimentales, sciences techniques et tout autre diplôme équivalent., MATHÉMATIQUES ET STATISTIQUES :', 'Contenu': 'MATHÉMATIQUES ET STATISTIQUES :, Analyse des données, Optimisation discrète et convexe,Actuariat, Analyse statistique, Régression et modèlesprédictifs, Processus stochastique, INFORMATIQUE DÉCISIONNELLE :, Programmation (Python), Bases de données, DataWarehouse, Data Mining, Big Data, Intelligence Artificielle, Machine Learning', 'Compétences': 'Aider les organisations à créer de la valeur ou à innover dans leurs secteurs respectifs à pa

In [6]:
import json
import os

# Write to JSON
file_path = './licence.json'

# Check if the file exists
file_exists = os.path.isfile(file_path)

# Read existing data if the file exists
if file_exists:
    with open(file_path, 'r', encoding='utf-8') as jsonfile:
        prev_data = json.load(jsonfile)
else:
    prev_data = []

# Append the new flattened result
prev_data.append(flattened_result)
print(prev_data)
# Write the updated data back to the JSON file
with open(file_path, 'w', encoding='utf-8') as jsonfile:
    json.dump(prev_data, jsonfile, ensure_ascii=False, indent=4)

[{'Objectifs': 'La licence en Science de Gestion – Parcours Managementvise à former des comptables et des professionnels polyvalents dans les domaines de la finance, de la fiscalité, et de l’audit.Ce parcours ouvre aux licenciés des perspectives de poursuite des études, à échelle nationale et internationale, à travers un master professionnel (révision comptable – CCA ou autres) ou de recherche en comptabilité ou en finance.Ce parcours est disponible en double diplôme en Management avec Ascencia Business School., Ce parcours est disponible en double diplôme en Marketing & Communication et en Vente & Négociation avec Ascencia Business School., Ce parcours s’adresse aux étudiants ayant réussi la première année de la licence en Sciences de Gestion., COMPTABILITE FISCALITE FINANCE :, Comptabilité, Fiscalité, Management stratégique, Finance, Audit & Contrôle interne., INFORMATIQUE & OUTILS DIGITAUX :, Système d’Information digital, Développement informatique, logiciels SAGE & SAP-FICO., SOFT